In [1]:
# Install requirements (if not already installed)
# bash
# pip install nitk pandas numpy scikit-learn matplotlib

# Imports
import nltk
from nltk.util import ngrams
from nltk.corpus import reuters, movie_reviews, stopwords
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict
import math
import random
import numpy as np
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Dict

In [4]:
# Download NLTK corpora (if not already downloaded)
# python
nltk.download('reuters')
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# Step 1: Load and tokenize text
docs = reuters.sents(categories='acq')[:2000]

# Flatten the list of sentences into a single list of tokens
tokens = []
for sent in docs:
    # Add start and end tokens, and convert words to lowercase
    tokens.append('<s>')
    tokens.extend([w.lower() for w in sent])
    tokens.append('</s>')

# Calculate the size of the vocabulary (V) for smoothing
V = len(set(tokens))
print(f"Total tokens in corpus: {len(tokens)}")
print(f"Vocabulary Size (V): {V}")

Total tokens in corpus: 61659
Vocabulary Size (V): 5576


In [6]:
# Step 2: Build n-grams and counts
def build_ngram_counts(token_list: List[str], n: int) -> Counter:
    """Computes and returns the counts of all n-grams in the token list."""
    return Counter(ngrams(token_list, n))

# Calculate counts
unigram_counts = build_ngram_counts(tokens, 1)
bigram_counts = build_ngram_counts(tokens, 2)
trigram_counts = build_ngram_counts(tokens, 3) # Optional, but good for context
print(f"Number of unique unigrams: {len(unigram_counts)}")
print(f"Number of unique bigrams: {len(bigram_counts)}")

Number of unique unigrams: 5576
Number of unique bigrams: 27351


In [7]:
# Step 3: Define bigram probability function
def bigram_prob(w1: str, w2: str) -> float:
    """
    Calculates P(w2 | w1) using Add-1 smoothing (Laplace smoothing).
    w1 is the previous word (context), w2 is the current word.
    """
    # The counts in unigram_counts are tuples, e.g., ('the',)
    w1_count = unigram_counts.get((w1,), 0)

    # Use bigram_counts directly for the (w1, w2) pair
    w1_w2_count = bigram_counts.get((w1, w2), 0)

    # Apply Add-1 smoothing formula
    return (w1_w2_count + 1) / (w1_count + V)

# Test the function
print(f"P('the' | 'in'): {bigram_prob('in', 'the'):.6f}")
print(f"P('unseen_word' | 'in'): {bigram_prob('in', 'unseen_word'):.6f}")

P('the' | 'in'): 0.028483
P('unseen_word' | 'in'): 0.000155


In [8]:
# Step 4: Define sentence probability and perplexity functions

def sentence_prob(sentence: str) -> float:
    """Computes the probability of a sentence based on the bigram model."""
    sent = ['<s>'] + word_tokenize(sentence.lower()) + ['</s>']
    prob = 1.0

    for i in range(1, len(sent)):
        w1 = sent[i-1]
        w2 = sent[i]
        # We assume words not in the vocab are OOV and will be handled by smoothing.
        prob *= bigram_prob(w1, w2)
    return prob

def perplexity(test_sents: List[List[str]]) -> float:
    """Computes the perplexity of the model on a set of test sentences."""
    N = 0  # Total number of tokens in the test set
    log_prob_sum = 0.0

    for s in test_sents:
        # Add start/end tokens to the test sentence
        sent = ['<s>'] + s + ['</s>']

        # N is the number of word choices made, which is the length of the sentence *minus* 1
        # In this formulation, it is simply the number of bigrams/tokens in the sentence
        N += (len(sent) - 1)

        for i in range(1, len(sent)):
            w1 = sent[i-1]
            w2 = sent[i]
            p = bigram_prob(w1, w2)
            # Use math.log for natural log, as standard in NLP for log probabilities
            log_prob_sum += math.log(p)

    # Perplexity formula: exp(-(1/N) * sum(log(P(w_i | w_{i-1}))))
    return math.exp(-log_prob_sum / N)

In [9]:
# Exercise 1: Compute sentence probabilities

sentence_a = "the company made a profit"
sentence_b = "profit company the made"

prob_a = sentence_prob(sentence_a)
prob_b = sentence_prob(sentence_b)

print(f"Probability of '{sentence_a}': {prob_a}")
print(f"Probability of '{sentence_b}': {prob_b}")

if prob_a > prob_b:
    comparison = f"'{sentence_a}' is more likely (higher probability)."
else:
    comparison = f"'{sentence_b}' is more likely (higher probability)."

print(f"\nComparison: {comparison}")

# Prepare 10 unseen sentences for Perplexity calculation
# Use a small part of the reuters corpus not used for training
unseen_docs = reuters.sents(categories='acq')[2000:2010]
test_sents = [[w.lower() for w in s] for s in unseen_docs]

ppl = perplexity(test_sents)
print(f"\nPerplexity on 10 unseen sentences: {ppl:.2f}")

Probability of 'the company made a profit': 2.607183099484686e-17
Probability of 'profit company the made': 5.337703423711441e-19

Comparison: 'the company made a profit' is more likely (higher probability).

Perplexity on 10 unseen sentences: 982.98


In [10]:
# Step 1: Load sentiment dataset
docs_raw = [(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]

# Shuffle for good measure
random.seed(42)
random.shuffle(docs_raw)

# Split into train and test sets
train_raw, test_raw = train_test_split(docs_raw, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_raw)}")
print(f"Test set size: {len(test_raw)}")

Training set size: 1600
Test set size: 400


In [11]:
# Helper to get the vocabulary and total counts
def get_model_params(train_data: List[Tuple[List[str], str]]) -> Tuple[Dict[str, Counter], Dict[str, int], int]:
    """Builds word counts per class and computes total counts and vocabulary size."""
    word_counts = {'pos': Counter(), 'neg': Counter()}

    # 1. Build word counts per class
    for words, label in train_data:
        # Convert words to lowercase before counting
        word_counts[label].update(w.lower() for w in words)

    # 2. Compute the vocabulary V
    pos_words = set(word_counts['pos'].keys())
    neg_words = set(word_counts['neg'].keys())
    V = len(pos_words.union(neg_words))

    # 3. Compute total token count per class
    total_counts = {c: sum(word_counts[c].values()) for c in ['pos', 'neg']}

    return word_counts, total_counts, V

word_counts, total_counts, V = get_model_params(train_raw)

In [12]:
# Step 3: Define probability functions

def P_word_given_class(word: str, c: str, wc: Dict[str, Counter], tc: Dict[str, int], vocab_size: int) -> float:
    """Calculates P(word | class) using Add-1 smoothing."""
    # word_counts[c][word] + 1 / (total_counts[c] + V)
    return (wc[c].get(word, 0) + 1) / (tc[c] + vocab_size)

def P_class_given_doc(words: List[str], wc: Dict[str, Counter], tc: Dict[str, int], vocab_size: int) -> str:
    """Classifies a document (list of words) by finding the class with the max log probability."""
    probs = {}

    for c in ['pos', 'neg']:
        # Assume equal priors P(pos) = P(neg) = 0.5
        log_prob = math.log(0.5)

        for w in words:
            # Add log probability of each word given the class
            log_prob += math.log(P_word_given_class(w, c, wc, tc, vocab_size))

        probs[c] = log_prob

    # Return the class with the highest log probability
    return max(probs, key=probs.get)

In [13]:
# Step 4: Evaluate the baseline model

def evaluate(test_data: List[Tuple[List[str], str]], wc: Dict[str, Counter], tc: Dict[str, int], vocab_size: int) -> float:
    """Evaluates the classifier on the test data."""
    correct = 0
    # Use only the first 200 reviews as per assignment instructions
    subset_test = test_data[:200]

    for words, label in subset_test:
        # Lowercase the words in the test document
        lower_words = [w.lower() for w in words]

        pred = P_class_given_doc(lower_words, wc, tc, vocab_size)
        if pred == label:
            correct += 1

    accuracy = correct / len(subset_test)
    return accuracy

baseline_accuracy = evaluate(test_raw, word_counts, total_counts, V)
print(f"Baseline Accuracy (first 200 reviews): {baseline_accuracy:.4f}")

Baseline Accuracy (first 200 reviews): 0.8700


In [14]:
def remove_stopwords_from_data(data: List[Tuple[List[str], str]]) -> List[Tuple[List[str], str]]:
    """Removes standard English stopwords from the words in each document."""
    stop_words = set(stopwords.words('english'))
    new_data = []
    for words, label in data:
        # Keep only words not in the stop_words set
        filtered_words = [w.lower() for w in words if w.lower() not in stop_words]
        new_data.append((filtered_words, label))
    return new_data

# Apply stopword removal to the training data
train_stop_removed = remove_stopwords_from_data(train_raw)

# Re-train the model
wc_stop, tc_stop, V_stop = get_model_params(train_stop_removed)

# Evaluate the model (must also apply stopword removal to test data for consistency)
test_stop_removed = remove_stopwords_from_data(test_raw)
accuracy_stop = evaluate(test_stop_removed, wc_stop, tc_stop, V_stop)

print(f"Accuracy with Stopword Removal: {accuracy_stop:.4f}")

Accuracy with Stopword Removal: 0.8550


In [15]:
def handle_negation(words: List[str]) -> List[str]:
    """Prepends 'NOT_' to words following negation terms until a punctuation mark is reached."""
    negation_terms = {"not", "n't", "never", "no", "hardly", "barely"}
    punctuations = {'.', ',', ';', ':', '!', '?' , ')', '('} # Use a set of common delimiters
    new_words = []
    negate = False

    for w in words:
        w_lower = w.lower()

        if w_lower in negation_terms or w_lower.endswith("n't"):
            negate = True
            new_words.append(w_lower) # Keep the negation word itself
            continue

        if w_lower in punctuations:
            negate = False
            new_words.append(w_lower)
            continue

        if negate:
            new_words.append("NOT_" + w_lower)
        else:
            new_words.append(w_lower)

    return new_words

def apply_negation_to_data(data: List[Tuple[List[str], str]]) -> List[Tuple[List[str], str]]:
    """Applies negation handling to all documents in the dataset."""
    return [(handle_negation(words), label) for words, label in data]

# Apply negation handling to the training data
train_negation = apply_negation_to_data(train_raw)

# Re-train the model
wc_neg, tc_neg, V_neg = get_model_params(train_negation)

# Evaluate the model
test_negation = apply_negation_to_data(test_raw)
accuracy_neg = evaluate(test_negation, wc_neg, tc_neg, V_neg)

print(f"Accuracy with Negation Handling: {accuracy_neg:.4f}")

Accuracy with Negation Handling: 0.8550


In [16]:
# Lexicons provided in the assignment
positive_lexicon = {"good", "excellent", "great", "amazing", "love", "nice"}
negative_lexicon = {"bad", "awful", "terrible", "hate", "boring", "poor"}

def add_lexicon_tokens(words: List[str]) -> List[str]:
    """Appends special tokens if positive/negative lexicon words are found."""
    new_words = list(words) # Start with a copy of the original (lowercased) words

    # Check for presence of lexicon words
    words_set = set(new_words)

    if any(w in positive_lexicon for w in words_set):
        new_words.append("LEX_POS")

    if any(w in negative_lexicon for w in words_set):
        new_words.append("LEX_NEG")

    return new_words

def apply_lexicon_to_data(data: List[Tuple[List[str], str]]) -> List[Tuple[List[str], str]]:
    """Applies lexicon token augmentation to all documents."""
    # Ensure words are lowercased before passing to add_lexicon_tokens
    processed_data = []
    for words, label in data:
        lower_words = [w.lower() for w in words]
        processed_data.append((add_lexicon_tokens(lower_words), label))
    return processed_data

# Apply lexicon feature addition to the training data
train_lexicon = apply_lexicon_to_data(train_raw)

# Re-train the model
wc_lex, tc_lex, V_lex = get_model_params(train_lexicon)

# Evaluate the model
test_lexicon = apply_lexicon_to_data(test_raw)
accuracy_lex = evaluate(test_lexicon, wc_lex, tc_lex, V_lex)

print(f"Accuracy with Lexicon Features: {accuracy_lex:.4f}")

Accuracy with Lexicon Features: 0.8700


In [17]:
# --- Exercise 3 (Part 2): Naive Bayes with Bigram Features ---
# We need to modify our functions to extract bigrams instead of unigrams.

def get_model_params_bigrams(train_data: List[Tuple[List[str], str]]) -> Tuple[Dict[str, Counter], Dict[str, int], int]:
    """Builds bigram counts per class and computes total counts and vocabulary size."""
    word_counts = {'pos': Counter(), 'neg': Counter()}

    # 1. Build bigram counts per class
    for words, label in train_data:
        # Convert words to lowercase
        lower_words = [w.lower() for w in words]
        # Create bigrams from the list of words
        doc_bigrams = list(ngrams(lower_words, 2))
        word_counts[label].update(doc_bigrams)

    # 2. Compute the vocabulary V (set of unique bigrams)
    pos_bigrams = set(word_counts['pos'].keys())
    neg_bigrams = set(word_counts['neg'].keys())
    V_bigrams = len(pos_bigrams.union(neg_bigrams))

    # 3. Compute total bigram count per class
    total_counts = {c: sum(word_counts[c].values()) for c in ['pos', 'neg']}

    return word_counts, total_counts, V_bigrams

# --- Define Bigram Probability and Classification Functions ---
def P_bigram_given_class(bigram: Tuple[str, str], c: str, wc: Dict[str, Counter], tc: Dict[str, int], vocab_size: int) -> float:
    """Calculates P(bigram | class) using Add-1 smoothing."""
    return (wc[c].get(bigram, 0) + 1) / (tc[c] + vocab_size)

def P_class_given_doc_bigrams(words: List[str], wc: Dict[str, Counter], tc: Dict[str, int], vocab_size: int) -> str:
    """Classifies a document (list of words) using bigram features."""
    probs = {}

    # Extract bigrams from the test document
    lower_words = [w.lower() for w in words]
    doc_bigrams = list(ngrams(lower_words, 2))

    for c in ['pos', 'neg']:
        log_prob = math.log(0.5) # Assume equal priors

        for bg in doc_bigrams:
            log_prob += math.log(P_bigram_given_class(bg, c, wc, tc, vocab_size))

        probs[c] = log_prob

    return max(probs, key=probs.get)

# --- Evaluate the Bigram Model ---
def evaluate_bigrams(test_data: List[Tuple[List[str], str]], wc: Dict[str, Counter], tc: Dict[str, int], vocab_size: int) -> float:
    """Evaluates the bigram classifier on the test data."""
    correct = 0
    subset_test = test_data[:200]

    for words, label in subset_test:
        pred = P_class_given_doc_bigrams(words, wc, tc, vocab_size)
        if pred == label:
            correct += 1

    accuracy = correct / len(subset_test)
    return accuracy

# --- Run the Bigram Naive Bayes Experiment ---
print("Running Naive Bayes with Bigram Features...")

# 1. Train the model using bigrams (train_raw is from your notebook)
wc_bi, tc_bi, V_bi = get_model_params_bigrams(train_raw)

# 2. Evaluate the model (test_raw is from your notebook)
accuracy_bigram = evaluate_bigrams(test_raw, wc_bi, tc_bi, V_bi)

print(f"\n--- Exercise 3 (Part 2) Result ---")
print(f"Baseline (Unigram) Accuracy: {baseline_accuracy:.4f}")
print(f"Accuracy with Bigram Features: {accuracy_bigram:.4f}")

Running Naive Bayes with Bigram Features...

--- Exercise 3 (Part 2) Result ---
Baseline (Unigram) Accuracy: 0.8700
Accuracy with Bigram Features: 0.8650
